# REFORMATTING AUDACITY FILES TO BE ABLE TO BE USED BY OPSO 0.12
OPSO contains helper functions to load raven files  
Audacity annotations must be modified to work with opensoundscapes as audacity generates TSV files instead of the expected CSV files

In [51]:

import pandas as pd
import os
import glob

### Prerequisite 2: Set paths (Absolute)
Make sure that your file structure looks like this
+ *"unique name for source of data (i.e. Macaulay, XC, field)"* ....... *folder*
    + csv files ....... *folder*
    + tsv files ....... *folder*
        + file1.wav
        + file2.wav
        + ...
        + fileN.wav


tsv files should contain all of your labels from audacity  
csv files should be COMPLETELY EMPTY, this will be populated with temporary csvs from this notebook

In [ ]:
pathtsv = "/home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/tsv files"
pathcsv = "/home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files"
audiofolder = "/home/dah238/Kauai-Amakihi (GIT)/Audio"

### Step 1: Convert audacity generated TSV files to CSV files

In [ ]:
os.makedirs(pathcsv, exist_ok=True)
def tsv_to_csv_pandas(tsv_file, csv_file):
    if os.path.getsize(tsv_file) == 0:
        print(f"Skipped empty file: {tsv_file}")
        return
    df = pd.read_csv(tsv_file, sep='\t')
    df.to_csv(csv_file, index=False) # index=False prevents writing the DataFrame index as a column


dir_list = os.listdir(pathtsv)

for item in dir_list:
    tsv_to_csv_pandas(f'{pathtsv}/{item}', f'{pathcsv}/{item[:-4]}.csv')
    print(f"Conversion complete: {item} converted to {item[:-4]}.csv using pandas")

Conversion complete: 5102.txt converted to 5102.csv using pandas
Conversion complete: 155986081.txt converted to 155986081.csv using pandas
Conversion complete: 5079.txt converted to 5079.csv using pandas
Conversion complete: 611565766.txt converted to 611565766.csv using pandas
Conversion complete: 156094151.txt converted to 156094151.csv using pandas
Conversion complete: 155551431.txt converted to 155551431.csv using pandas
Conversion complete: 5112.txt converted to 5112.csv using pandas
Conversion complete: 155551441.txt converted to 155551441.csv using pandas
Conversion complete: 5118.txt converted to 5118.csv using pandas
Conversion complete: 155988551.txt converted to 155988551.csv using pandas
Conversion complete: 5116.txt converted to 5116.csv using pandas
Conversion complete: 20582.txt converted to 20582.csv using pandas
Conversion complete: 156091191.txt converted to 156091191.csv using pandas
Conversion complete: 5035.txt converted to 5035.csv using pandas
Conversion complet

IsADirectoryError: [Errno 21] Is a directory: '/home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/tsv files/csv files'

### Step 2: Format the CSVs to include file path in the CSV

In [54]:
def formatting_csv(csv_file,audiofolder):
    # Read the CSV and rename columns
    df = pd.read_csv(csv_file, header=None)
    df.rename(columns={0: 'start_time', 1: 'end_time', 2: 'annotation'}, inplace=True)

    # Audio folder and file match logic
    audio_extensions = ['wav', 'm4a', 'mp3']
    audio_folder = '/home/dah238/Kauai-Amakihi (GIT)/Audio'
    base_name = os.path.splitext(os.path.basename(csv_file))[0]

    matched_file = None
    for ext in audio_extensions:
        pattern = os.path.join(audio_folder, '**', f'*{base_name}*.{ext}')
        matches = glob.glob(pattern, recursive=True)

        for m in matches:
            audio_base = os.path.splitext(os.path.basename(m))[0].lower()
            if base_name.lower() in audio_base:
                matched_file = m
                break
        if matched_file:
            break

    # Handle matched/unmatched audio
    if matched_file:
        relative_path = os.path.abspath(matched_file)
    else:
        print(f"⚠️ No match found for {base_name}")
        relative_path = os.path.join(audio_folder, f"{base_name}.UNKNOWN")

    df['file'] = relative_path

    # Define output directory
    output_dir = pathcsv
    os.makedirs(output_dir, exist_ok=True)

    # Save file
    output_path = os.path.join(output_dir, f"{base_name}_reform.csv")
    df.to_csv(output_path, index=False)
    print(f"✅ Saved reformatted CSV: {output_path}")


# === Main section ===

if not os.path.exists(pathcsv):
    raise FileNotFoundError(f"❌ Directory not found: {pathcsv}")

dir_list = [f for f in os.listdir(pathcsv) if f.endswith('.csv')]

if not dir_list:
    print(f"⚠️ No CSV files found in {pathcsv}")
else:
    for item in dir_list:
        full_path = os.path.join(pathcsv, item)
        formatting_csv(full_path,audiofolder)
        print(f"Conversion complete: {item}")

✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155971191_reform.csv
Conversion complete: 155971191.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155551481_reform.csv
Conversion complete: 155551481.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/518079991_reform.csv
Conversion complete: 518079991.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/156091191_reform.csv
Conversion complete: 156091191.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5113_reform.csv
Conversion complete: 5113.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5118_reform.csv
Conversion complete: 5118.csv
✅ Saved reformatted CSV: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5109_reform.csv
Conversion complete: 5109.csv
✅ Saved reforma

### Optional Step: Remove all old CSV files

In [55]:
# === Delete Non-_reform CSVs ===
print("\n🧹 Cleaning up non-reformatted CSV files...")

deleted_count = 0
for root, dirs, files in os.walk(pathcsv):
    for file in files:
        if file.endswith('.csv') and not file.endswith('_reform.csv'):
            full_path = os.path.join(root, file)
            try:
                os.remove(full_path)
                deleted_count += 1
                print(f"🗑️ Deleted: {full_path}")
            except Exception as e:
                print(f"❌ Error deleting {full_path}: {e}")

print(f"\n✅ Cleanup complete. {deleted_count} original CSV file(s) deleted.")


🧹 Cleaning up non-reformatted CSV files...
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155971191.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155551481.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/518079991.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/156091191.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5113.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5118.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5109.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/518079981.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155997081.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/611565766.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv f

### Step 3: Combine all Reformatted (with file path column and headers) CSVs into one combined output

In [ ]:
pattern = os.path.join(pathcsv, '*_reform.csv')
reform_files = glob.glob(pattern)

if not reform_files:
    print(f"⚠️ No reformatted CSV files found in {pathcsv}.")
else:
    df_list = []
    for f in reform_files:
        try:
            df = pd.read_csv(f)
            df_list.append(df)
            print(f"📄 Added: {os.path.basename(f)}")
        except Exception as e:
            print(f"❌ Error reading {f}: {e}")

    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        combined_df = combined_df.rename(columns={"file": "audio_file"})


        save_path = '/home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/combined_output.csv'
        combined_df.to_csv(save_path, index=False)

        print(f"\n✅ Combined CSV saved to: {save_path}")
        print(f"📊 Total rows: {len(combined_df)} across {len(df_list)} files")
    else:
        print("⚠️ No valid data frames were loaded. Combined file not created.")

📄 Added: 155997081_reform.csv
📄 Added: 5102_reform.csv
📄 Added: 5037_reform.csv
📄 Added: 101775931_reform.csv
📄 Added: 155551411_reform.csv
📄 Added: 5832_reform.csv
📄 Added: 156091191_reform.csv
📄 Added: 5056_reform.csv
📄 Added: 5821_reform.csv
📄 Added: 20582_reform.csv
📄 Added: 518079991_reform.csv
📄 Added: 5028_reform.csv
📄 Added: 155971191_reform.csv
📄 Added: 156004801_reform.csv
📄 Added: 5109_reform.csv
📄 Added: 5113_reform.csv
📄 Added: 155551481_reform.csv
📄 Added: 611565766_reform.csv
📄 Added: 615347576_reform.csv
📄 Added: 156094711_reform.csv
📄 Added: 5112_reform.csv
📄 Added: 155983261_reform.csv
📄 Added: 155551451_reform.csv
📄 Added: 518079981_reform.csv
📄 Added: 155551441_reform.csv
📄 Added: 487421021_reform.csv
📄 Added: 156092401_reform.csv
📄 Added: 155988551_reform.csv
📄 Added: 155551431_reform.csv
📄 Added: 156094151_reform.csv
📄 Added: 5116_reform.csv
📄 Added: 554099931_reform.csv
📄 Added: 5837_reform.csv
📄 Added: 5032_reform.csv
📄 Added: 5118_reform.csv
📄 Added: 156002671_

### Optional Step: Remove all formatted CSVs now that combined_output.csv has been created

In [57]:
# === Delete Non-_reform CSVs ===
print("\n🧹 Cleaning up reformatted CSV files...")

deleted_count = 0
for root, dirs, files in os.walk(pathcsv):
    for file in files:
        if file.endswith('_reform.csv'):
            full_path = os.path.join(root, file)
            try:
                os.remove(full_path)
                deleted_count += 1
                print(f"🗑️ Deleted: {full_path}")
            except Exception as e:
                print(f"❌ Error deleting {full_path}: {e}")

print(f"\n✅ Cleanup complete. {deleted_count} original CSV file(s) deleted.")


🧹 Cleaning up reformatted CSV files...
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155997081_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5102_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5037_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/101775931_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/155551411_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5832_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/156091191_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5056_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/5821_reform.csv
🗑️ Deleted: /home/dah238/Kauai-Amakihi (GIT)/Annotations/macaulay/csv files/20582_reform.csv
🗑️ Deleted: /home/d